# Algorithm to find the examples from Thm 2 [Kurtz, 1992]

Tested using Julia 1.9.4, Oscar 1.0.5, IntervalArithmetic 0.22.

Loading packages:

In [74]:
using Oscar, IntervalArithmetic 

Defining $\mathbb{R}[t]$:

In [75]:
RR = RealField()
R, t = RR[:t] # polynomial ring

(Univariate polynomial ring in t over RR, t)

### find_Kurtz_example

In [76]:
function find_Kurtz_example(epsilon::BigFloat, degree::Int)
    """
    Returns example constructed from given degree and epsilon, as in the proof of Theorem 2 in the article [Kurtz, 1992].
    """
    epsilon = BigFloat(epsilon)
    # We want to find a degree (degree-1) polynomial, which is a example in degree (n-1) and with epsilon/2.
    # If degree = 2, we need to find such a degree 2 polynomial with the same epsilon. 
    #       (Note: this only happens, if the original degree is 2 and there is no recursion.)
    # Else, the original degree is more than 2:
        # If degree = 3, we need to find such a degree 2 polynomial with epsilon/2.
        # Else, we recurse.
    if degree == 2 # original degree = 2 (no recursion)
        return find_deg2_Kurtz_example(epsilon)
    else # original degree > 2
        if degree == 3
            pol_deg_n_1 = find_deg2_Kurtz_example(BigFloat(epsilon/2))
        else
            pol_deg_n_1 = find_Kurtz_example(BigFloat(epsilon/2), Int(degree-1))
        end
        # Find f_mu from pol_deg_n_1
        return find_f_mu(pol_deg_n_1, epsilon)
    end
end

find_Kurtz_example (generic function with 1 method)

### find_deg2_Kurtz_example

In [77]:
function find_deg2_Kurtz_example(epsilon::BigFloat)
    """
    Returns base case (degree 2) example constructed from given epsilon. 
    This function is called as part of the ´find_Kurtz_example´ algorithm.
    """
    epsilon = BigFloat(epsilon)
    # As we pick a=b=1, we only need to find c.
    # The upper limit on c depends on the value of epsilon:
    if (epsilon >= 4)
        c = 1 # as we just need c > 1/4
    else # if (epsilon < 4)
        less_than_c = convert(BigFloat, 1/4)
        greater_than_c = convert(BigFloat, -(1/(epsilon - 4)))
        possible_values = IntervalArithmetic.bareinterval(less_than_c, greater_than_c)
        #less_than_c = 1/4
        #greater_than_c = -(1/(epsilon - 4))
        c = IntervalArithmetic.mid(possible_values, 0.5)
    end
    # The degree 2 polynomial:
    return R(BigFloat[c,1,1])
end

find_deg2_Kurtz_example (generic function with 1 method)

### find_f_mu

In [78]:
function find_f_mu(pol_deg_n_1::RealPoly, epsilon::BigFloat)
    """
    Returns next polynomial, when given a polynomial of degree one less.
    This function is called as part of the ´find_Kurtz_example´ algorithm.
    """
    epsilon = BigFloat(epsilon)
    # Starting value of mu:
    mu = 1

    while !(check_mu(pol_deg_n_1, epsilon, mu))
        mu = mu + 1
    end
    return (mu*t+1)*pol_deg_n_1
end

find_f_mu (generic function with 1 method)

### check_mu

In [79]:
function check_mu(pol_deg_n_1::RealPoly, epsilon::BigFloat, mu)
    """
    Determines if the condition 'S(f,i)>4-epsilon, for all i=1,...,n-1' is satisfied.
    This function is called as part of the ´find_Kurtz_example´ algorithm.
    """
    epsilon = BigFloat(epsilon)
    coeffs = collect(coefficients(pol_deg_n_1)) # note: index 1 is constant term, index n+1 is n'th term
    degree_n = length(pol_deg_n_1)

    # First consider the case i=1:
    S = (mu*coeffs[0+1]^2 + 2*coeffs[0+1]*coeffs[1+1] + (1/mu)*coeffs[1+1]^2)/(coeffs[0+1]*coeffs[1+1]+(1/mu)*coeffs[0+1]*coeffs[1+1])
    if S <= (4-epsilon)
        return false
    end
    # Next, the case i=2,...,n-2:
    for i in 2:(degree_n-2)
        S = (coeffs[(i-1)+1]^2 + (1/mu)*2*coeffs[(i-1)+1]*coeffs[i+1] + (1/mu^2)*coeffs[i+1]^2) / (coeffs[i+1]*coeffs[(i-2)+1]+(1/mu)*(coeffs[(i-2)+1]*coeffs[(i+1)+1]+coeffs[i+1]*coeffs[(i-1)+1]) + (1/mu^2)*coeffs[(i-1)+1]*coeffs[(i+1)+1])
        if S <= (4-epsilon)
            return false
        end
    end
    # Lastly, the case i=n-1:
    S = (coeffs[(degree_n-2)+1]^2 + (1/mu)*2*coeffs[(degree_n-2)+1]*coeffs[(degree_n-1)+1] + (1/mu^2)*coeffs[(degree_n-1)+1]^2) / (coeffs[(degree_n-1)+1]*coeffs[(degree_n-3)+1]+(1/mu)*coeffs[(degree_n-1)+1]*coeffs[(degree_n-2)+1])
    if S <= (4-epsilon)
        return false
    end
    return true
end

check_mu (generic function with 1 method)

### Function to check if a polynomial and some epsilon satisfy the conditions

In [80]:
function check_conditions(pol::RealPoly, epsilon::BigFloat)
    """
    Determines if all conditions are satisfied:
        - All coefficients are positive.
        - S(P,i)>4-epsilon, for all i=1,...,n-1
        - The polynomial has some non-real roots.
    Input:
        - RealPoly pol
        - BigFloat epsilon
    """
    epsilon = BigFloat(epsilon)
    coeffs = collect(coefficients(pol)) # note: index 1 is constant term, index n+1 is n'th term
    pol_degree = Oscar.degree(pol)
    
    # Check that all coefficients are positive:
    for a in coeffs
        if a <= 0
            return (false, "Not all coefficients are positive.")
        end
    end
    
    # Check that S(f,i)>(4-epsilon) for all i:
    for i in 1:(pol_degree - 1)
        S = (coeffs[i+1]^2)/(coeffs[(i-1)+1]*coeffs[(i+1)+1])
        if S <= (4-epsilon)
            return (false, "S(pol, " * string(i) * ")<=(4-epsilon)")
        end
    end

    # Check that the polynomial has some non-real roots:
    coeffs = collect(coefficients(pol))
    CC = ComplexField()
    C, z = CC[:z]
    fC = C(coeffs)
    complex_roots = roots(fC)
    number_of_real_roots = 0
    for root in complex_roots
        if isreal(root)
            number_of_real_roots += 1
        end
    end
    if number_of_real_roots == pol_degree
        return (false, "All roots are real.")
    end

    return (true, "All conditions are satisfied.")
end

check_conditions (generic function with 1 method)

## Example

Note: for some reason, you need to write BigFloat(epsilon) for it to work. I am unsure why. I am working on figuring this out.

Finding polynomial for $n=5$, $\epsilon = 1/2$:

In [81]:
f = find_Kurtz_example(BigFloat(1/2),5)

2086080.0000000000000*t^5 + 2155756.0000000000000*t^4 + [595960.047619047619 +/- 1.89e-13]*t^3 + [18183.2460317460317 +/- 5.07e-14]*t^2 + [158.4900793650793651 +/- 6.14e-17]*t + [0.2519841269841269841 +/- 5.24e-20]

Same polynomial but in $\mathbb{C}[t]$ (so we can easily determine roots):

In [82]:
# Vector of coefficients of f:
coeffs = collect(coefficients(f))
# Defining complex polynomial ring:
CC = ComplexField()
C, z = CC[:z]
# Complex version of polynomial: (exact same polynomial, but changing base field)
fC = C(coeffs)

2086080.0000000000000*z^5 + 2155756.0000000000000*z^4 + [595960.047619047619 +/- 1.89e-13]*z^3 + [18183.2460317460317 +/- 5.07e-14]*z^2 + [158.4900793650793651 +/- 6.14e-17]*z + [0.2519841269841269841 +/- 5.24e-20]

Finding (approximated) roots of the polynomial:

In [83]:
roots(fC)

5-element Vector{ComplexFieldElem}:
 [-0.01886792 +/- 5.07e-9]
 [-0.012500000 +/- 3.77e-10]
 [-0.0020325203 +/- 3.92e-11]
 [-0.50000000 +/- 8.70e-9] + [-0.04454354 +/- 9.32e-9]*im
 [-0.50000000 +/- 8.64e-9] + [0.0445435 +/- 5.00e-8]*im

So our polynomial has some non-real roots as expected.

### Checking if we can remove the decimal parts

Preliminary check with the new function:

In [84]:
check_conditions(f, BigFloat(1/2))

(true, "All conditions are satisfied.")

Removing the error bounds/intervals:

In [85]:
g = 2086080*t^5 + 2155756*t^4 + 595960.047619047619*t^3 + 18183.2460317460317*t^2 + 158.4900793650793651*t + 0.2519841269841269841
check_conditions(g, BigFloat(1/2))

(true, "All conditions are satisfied.")

Rounding down (constant term rounded to 2 decimals and the rest to integers):

In [86]:
p = 2086080*t^5 + 2155756*t^4 + 595960*t^3 + 18183*t^2 + 158*t + 0.25
print(check_conditions(p, BigFloat(1/2)))

(true, "All conditions are satisfied.")

Polynomial multiplied by 4:

In [89]:
println(4*p)
print(check_conditions(4*p,BigFloat(1/2)))

8344320.0000000000000*t^5 + 8623024.0000000000000*t^4 + 2383840.0000000000000*t^3 + 72732.000000000000000*t^2 + 632.00000000000000000*t + 1
(true, "All conditions are satisfied.")